In [1]:
from __future__ import division
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd 
import gspread
from google.oauth2.service_account import Credentials
from celery import Celery
import io
import math
from io import BytesIO
from PIL import Image
import base64 
from celery.utils.log import get_task_logger
import os
import shutil

logger = get_task_logger(__name__)

In [29]:
INPUT_FILE = './countries.csv'
path = './'
image_ext = '.png'

df = pd.read_csv(INPUT_FILE)
countries = dict(zip(df.Country.str.lower(), df.l2.str.lower()))

scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]
credentials = Credentials.from_service_account_file('service_account.json',scopes=scopes)

gc = gspread.authorize(credentials)

In [30]:
from webdriver_manager.chrome import ChromeDriverManager

chrome_options = Options()  
chrome_options.add_argument("--headless") 
#chrome_options.add_argument("--no-sandbox"),
#chrome_options.add_argument("--disable-infobars"),# // https://stackoverflow.com/a/43840128/1689770
#chrome_options.add_argument("--disable-dev-shm-usage"),# // https://stackoverflow.com/a/50725918/1689770
#chrome_options.add_argument("--disable-browser-side-navigation"),# // https://stackoverflow.com/a/49123152/1689770
chrome_options.add_argument("--disable-gpu"),# // https://stackoverflow.com/questions/51959986/how-to-solve-selenium-chromedriver-timed-out-receiving-message-from-renderer-exc
#chrome_options.add_argument("--disable-features=VizDisplayCompositor")#  
url = os.environ.get("SELENIUM_SERVER_URL", "http://localhost:4444/wd/hub")
try:
    driver = webdriver.Remote(command_executor=url,options=chrome_options)
except Exception as e:
    print("Exception", e)
    driver.quit()

In [31]:
def _make_query(word,language,country):
    base_url = 'https://www.google.com/search?q='
    query = word.replace(' ','%20')
    lr="&lr=lang_xx".replace('xx',language.lower())
    cr="&cr=countryXX".replace('XX',country.upper())
    hl="&hl=xx".replace('xx',language.lower())
    return base_url+query+lr+cr+hl

In [32]:
def long_slice(img, slice_size = 1080):
    logger.info('long_slice')
    """slice an image into parts slice_size tall"""
    width, height = img.size
    upper = 0
    left = 0
    slices = int(math.ceil(height/slice_size))

    count = 1
    im_slices = []
    for _ in range(slices):
        #if we are at the end, set the lower bound to be the bottom of the image
        if count == slices:
            lower = height
        else:
            lower = int(count * slice_size)  

        bbox = (left, upper, width, lower)
        working_slice = img.crop(bbox)
        upper += slice_size
        #save the slice
        im_slices.append(working_slice)
        count +=1
    
    return im_slices

In [33]:
from PIL import Image
import io

def image_to_byte_array(image: Image):
    buffer = io.BytesIO()
    image.save(buffer, format='png')
    return buffer

In [34]:
def get_screenshot_pil(driver, link):
    driver.get(link)
    S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
    driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment
    screen_base64 = driver.find_element(by=By.TAG_NAME, value='body').screenshot_as_base64
    image = Image.open(io.BytesIO(base64.decodebytes(bytes(screen_base64, "utf-8"))))
    return image

In [35]:
def crawl(task_id,link,language,country,driver, df):
    images = {}
    for index, row in df.iterrows():
        word = row['Words'].strip()
        images[word] = {}
        link = _make_query(word,language,country)
        image = get_screenshot_pil(driver, link)
        im_slices = long_slice(image)
        print(index, link, len(im_slices))
        
        #images_dir = f'./screenshots/{task_id}/{word}/{language}/{country}/'
        #os.makedirs(os.path.dirname(images_dir), exist_ok=True)
        
        for slice_ind, aslice in enumerate(im_slices):
            image_name = 'Images'+str(slice_ind)+image_ext
            images[word][image_name] = image_to_byte_array(aslice)
            #image_path = images_dir+image_name
            #aslice.save(image_path)
            df.at[index, image_name] = '' # aslice #str(image_path)
    #worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    
    return (df,images)
    # ==== DELETING 
    #logger.info("Done")
    #shutil.rmtree('./screenshots/{task_id}/')
    
    #driver.quit()    
    #return True

In [36]:
link = 'https://docs.google.com/spreadsheets/d/1V6I9IvHQpsKr_xw2sVV5dOGxq4WeifKSOe-iM3M5RZ8/edit#gid=0'
sh = gc.open_by_url(link)
worksheet = sh.get_worksheet(0)
df = pd.DataFrame(worksheet.get_all_records())

df_output, images = crawl(
    task_id = 'taskid123',
    link = link,
    language = 'fr',
    country = 'de',
    driver = driver,
    df = df
)

0 https://www.google.com/search?q=Skirt&lr=lang_fr&cr=countryDE&hl=fr 3
1 https://www.google.com/search?q=Bikini&lr=lang_fr&cr=countryDE&hl=fr 3
2 https://www.google.com/search?q=Dress%20pants&lr=lang_fr&cr=countryDE&hl=fr 3
3 https://www.google.com/search?q=Jumper&lr=lang_fr&cr=countryDE&hl=fr 3
4 https://www.google.com/search?q=Sneakers&lr=lang_fr&cr=countryDE&hl=fr 3
5 https://www.google.com/search?q=Hoodie&lr=lang_fr&cr=countryDE&hl=fr 3
6 https://www.google.com/search?q=Vest&lr=lang_fr&cr=countryDE&hl=fr 3
7 https://www.google.com/search?q=High%20heels&lr=lang_fr&cr=countryDE&hl=fr 3
8 https://www.google.com/search?q=Flip%20flops&lr=lang_fr&cr=countryDE&hl=fr 3
9 https://www.google.com/search?q=Handbag&lr=lang_fr&cr=countryDE&hl=fr 3
10 https://www.google.com/search?q=Tank%20top&lr=lang_fr&cr=countryDE&hl=fr 3
11 https://www.google.com/search?q=Singlet&lr=lang_fr&cr=countryDE&hl=fr 3
12 https://www.google.com/search?q=Boots&lr=lang_fr&cr=countryDE&hl=fr 3
13 https://www.google.com

108 https://www.google.com/search?q=Socks&lr=lang_fr&cr=countryDE&hl=fr 3
109 https://www.google.com/search?q=Diaper&lr=lang_fr&cr=countryDE&hl=fr 3
110 https://www.google.com/search?q=Singlet&lr=lang_fr&cr=countryDE&hl=fr 3
111 https://www.google.com/search?q=Shoes&lr=lang_fr&cr=countryDE&hl=fr 3
112 https://www.google.com/search?q=Safety%20pin&lr=lang_fr&cr=countryDE&hl=fr 3
113 https://www.google.com/search?q=T-shirt&lr=lang_fr&cr=countryDE&hl=fr 3
114 https://www.google.com/search?q=Dress&lr=lang_fr&cr=countryDE&hl=fr 3
115 https://www.google.com/search?q=Singlet&lr=lang_fr&cr=countryDE&hl=fr 3
116 https://www.google.com/search?q=Cap&lr=lang_fr&cr=countryDE&hl=fr 3
117 https://www.google.com/search?q=One-piece%20bathing%20suit&lr=lang_fr&cr=countryDE&hl=fr 3
118 https://www.google.com/search?q=Hawaiian%20shirt&lr=lang_fr&cr=countryDE&hl=fr 3
119 https://www.google.com/search?q=Sunglasses&lr=lang_fr&cr=countryDE&hl=fr 3
120 https://www.google.com/search?q=Swim%20trunks&lr=lang_fr&cr

214 https://www.google.com/search?q=Cotton%20swab&lr=lang_fr&cr=countryDE&hl=fr 3
215 https://www.google.com/search?q=Tweezers&lr=lang_fr&cr=countryDE&hl=fr 3
216 https://www.google.com/search?q=Scissors&lr=lang_fr&cr=countryDE&hl=fr 3
217 https://www.google.com/search?q=Mouthwash&lr=lang_fr&cr=countryDE&hl=fr 3
218 https://www.google.com/search?q=Dental%20floss&lr=lang_fr&cr=countryDE&hl=fr 3
219 https://www.google.com/search?q=Comb&lr=lang_fr&cr=countryDE&hl=fr 3
220 https://www.google.com/search?q=Hair%20spray&lr=lang_fr&cr=countryDE&hl=fr 3
221 https://www.google.com/search?q=Hair%20clip&lr=lang_fr&cr=countryDE&hl=fr 3
222 https://www.google.com/search?q=Hair%20band&lr=lang_fr&cr=countryDE&hl=fr 3
223 https://www.google.com/search?q=Hairbrush&lr=lang_fr&cr=countryDE&hl=fr 3
224 https://www.google.com/search?q=Bobby%20pin&lr=lang_fr&cr=countryDE&hl=fr 3
225 https://www.google.com/search?q=Hair%20dryer&lr=lang_fr&cr=countryDE&hl=fr 3
226 https://www.google.com/search?q=Hair%20gel&lr=

In [37]:
import xlsxwriter
writer_excel = pd.ExcelWriter("bank_summ.xlsx", engine= "xlsxwriter")
df_output.to_excel(writer_excel, index=False, sheet_name= "Sheet1")
workbook = writer_excel.book
worksheet = writer_excel.sheets["Sheet1"]

cell_format = workbook.add_format()
cell_format.set_align('center')
cell_format.set_align('top')

worksheet.set_default_row(400)
worksheet.set_row(0, 20) 
worksheet.set_column("A:A", 20, cell_format)
worksheet.set_column("B:E", 80, cell_format)
#img=Image.open("Money Density.png")
#new_img= img.resize((512,300))
#new_img.save(loc+"new_img.png")
#worksheet.insert_image(1, df.shape[1]+2, loc+"new_img.png")

0

In [38]:
cols = df_output.columns.tolist()
row_index = 0
for word, data in images.items():
    row_index += 1
    for image_name, bytearr in data.items():
        col_index = cols.index(image_name)
        img_data =  {'image_data': images[word][image_name],'y_scale':0.5, 'x_scale':0.5}
        worksheet.insert_image(row_index, col_index, image_name, img_data)

In [39]:
writer_excel.save()

In [ ]:
# OLD
def row_iterator(row, row_index, cols, ws):
    row_index = row_index + 1
    for ind, image_path in enumerate(cols):
        if 'Images' in image_path and isinstance(row[image_path], str):
            col_index = ind +1
            ws.insert_image(row_index, col_index, row[image_path])
            ws.insert_image('B5', 'image.png', {'image_data': image_data, 'x_scale': 0.5, 'y_scale': 0.5})